In [77]:
import numpy as np
import pandas as pd
import scipy.stats as stats


from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge
from sklearn.model_selection import KFold

import statsmodels.api as sm

import matplotlib.pyplot as plt
plt.style.use('ggplot')

%matplotlib inline

In [78]:
data = pd.read_csv('forecast_HIV_infections/data/x_train.csv', index_col=0)
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2355 entries, 0 to 2354
Data columns (total 38 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   county_code        2355 non-null   int64  
 1   COUNTY             2355 non-null   object 
 2   STATEABBREVIATION  2355 non-null   object 
 3   YEAR               2355 non-null   int64  
 4   AMAT_fac           2355 non-null   float64
 5   HIVdiagnoses       2355 non-null   float64
 6   HIVprevalence      2355 non-null   float64
 7   MH_fac             2355 non-null   float64
 8   Med_AMAT_fac       2355 non-null   float64
 9   Med_MH_fac         2355 non-null   float64
 10  Med_SA_fac         2355 non-null   float64
 11  Med_SMAT_fac       2355 non-null   float64
 12  Med_TMAT_fac       2355 non-null   float64
 13  PLHIV              2355 non-null   float64
 14  Population         2355 non-null   float64
 15  SA_fac             2355 non-null   float64
 16  SMAT_fac           2355 

In [9]:
X = data.drop(columns=['COUNTY','STATEABBREVIATION','partD30dayrxrate']).values
y = pd.read_csv('forecast_HIV_infections/data/y_train.csv', index_col=0).values
X

array([[5.1570e+04, 2.0150e+03, 0.0000e+00, ..., 1.0600e+01, 7.0250e+03,
        1.2100e+01],
       [1.6013e+04, 2.0150e+03, 0.0000e+00, ..., 1.0800e+01, 9.2580e+03,
        1.7600e+01],
       [2.0900e+03, 2.0150e+03, 0.0000e+00, ..., 8.0000e+00, 3.5844e+04,
        1.4300e+01],
       ...,
       [1.6039e+04, 2.0150e+03, 0.0000e+00, ..., 1.7000e+01, 9.6820e+03,
        1.5100e+01],
       [5.0110e+03, 2.0150e+03, 0.0000e+00, ..., 2.9400e+01, 4.7200e+03,
        2.0000e+01],
       [3.9139e+04, 2.0150e+03, 0.0000e+00, ..., 1.6300e+01, 4.8211e+04,
        1.1900e+01]])

In [10]:
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.preprocessing import StandardScaler


class XyScaler(BaseEstimator, TransformerMixin):
    """Standardize a training set of data along with a vector of targets."""

    def __init__(self):
        self.X_scaler = StandardScaler()
        self.y_scaler = StandardScaler()
        
    def fit(self, X, y, *args, **kwargs):
        """Fit the scaler to data and a target vector."""
        self.X_scaler.fit(X)
        self.y_scaler.fit(y.reshape(-1, 1))
        return self
    
    def transform(self, X, y, *args, **kwargs):
        """Transform a new set of data and target vector."""
        return (self.X_scaler.transform(X),
                self.y_scaler.transform(y.reshape(-1, 1)).flatten())

    def inverse_transform(self, X, y, *args, **kwargs):
        """Tranform from a scaled representation back to the original scale."""
        return (self.X_scaler.inverse_transform(X),
                self.y_scaler.inverse_transform(y.reshape(-1, 1)).flatten())

In [75]:
def cross_val(X, y, model, n_folds=10, random_seed=154, scale=True):
    kf = KFold(n_splits=n_folds, shuffle=True, random_state=random_seed)
    test_cv_errors, train_cv_errors = np.empty(n_folds), np.empty(n_folds)
    scaler = XyScaler()

    for idx, (train_idx, valid_idx) in enumerate (kf.split(X)):
        
        if scale:
            scaler.fit(X[train_idx], y[train_idx])
            std_X_train, std_y_train = scaler.transform(X[train_idx], y[train_idx])
            std_X_valid, std_y_valid = scaler.transform(X[valid_idx], y[valid_idx])
        else:
            std_X_train, std_y_train = X[train_idx], y[train_idx]
            std_X_valid, std_y_valid = X[valid_idx], y[valid_idx]
            
        model.fit(std_X_train,std_y_train)
        y_train_pred = model.predict(std_X_train)
        y_test_pred = model.predict(std_X_valid)
        
        train_cv_errors[idx] = mean_squared_error(std_y_train, y_train_pred)
        test_cv_errors[idx] = mean_squared_error(std_y_valid, y_test_pred)

    return train_cv_errors, test_cv_errors


In [76]:
def train_at_various_alphas(X, y, model, alphas, n_folds=10, **kwargs):
    
    cv_errors_train = np.empty(shape=(n_folds, len(alphas)))
    cv_errors_test = np.empty(shape=(n_folds, len(alphas)))
    
    for idx, alpha in enumerate(alphas):
        train_cv_errors, test_cv_errors = cross_val(X, y, model(alpha=alpha), n_folds=n_folds)
        cv_errors_train[:,idx] = train_cv_errors
        cv_errors_test[:,idx] = test_cv_errors
        
    return cv_errors_train, cv_errors_test



In [37]:
model = LinearRegression()
mse_train, mse_test = cross_val(X, y, model, scale=False)

In [38]:
mse_train

array([255.4840547 , 258.48507275, 256.89610909, 253.67174044,
       251.05565773, 252.39365854, 256.81677787, 237.06446542,
        36.9421193 , 252.72960526])

In [26]:
model = LinearRegression()
mse_train, mse_test = cross_val(X, y, model, scale=True)

In [27]:
mse_train

array([0.70209315, 0.69962173, 0.70394835, 0.69906187, 0.68531315,
       0.69512365, 0.69589307, 0.6494347 , 0.41769809, 0.69220799])

In [28]:
model = Ridge(alpha=0.5)
mse_train, mse_test = cross_val(X, y, model, scale=True)

In [29]:
mse_train

array([0.70250837, 0.69992407, 0.70437074, 0.69942061, 0.68582102,
       0.69556185, 0.69622042, 0.64995537, 0.41796317, 0.69252962])

In [73]:
alphas = [1,10,100,100,1000,10000]
mse_train, mse_test = train_at_various_alphas(X, y, Ridge, alphas)

In [74]:
mse_train

array([0.66462778, 0.67105382, 0.72921933, 0.72921933, 0.80469303,
       0.90129752])